In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [2]:
#creating dataframes of testing and trainig data
train= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
#let us see how the training and testing data look
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
#splitting the data into training and validation sets
from sklearn.model_selection import train_test_split
X=train.drop("SalePrice",axis=1)
y=train.SalePrice
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9, test_size=0.1,
                                                      random_state=0)

In [5]:
y.isnull().sum()

0

so there is no null values in the target variable, we dont need to delete any record

In [6]:
print(X_train.shape) 
print(X_valid.shape)

(1314, 80)
(146, 80)


Checking for missing values, findings columns with greater number of missing values

In [7]:
B=[]
for col in X_train.columns:
    if(X_train[col].isnull().sum()>10):
        B.append(col)
X_train[B].isnull().sum()

LotFrontage      233
Alley           1233
BsmtQual          32
BsmtCond          32
BsmtExposure      33
BsmtFinType1      32
BsmtFinType2      33
FireplaceQu      621
GarageType        74
GarageYrBlt       74
GarageFinish      74
GarageQual        74
GarageCond        74
PoolQC          1308
Fence           1070
MiscFeature     1263
dtype: int64

let us drop the columns which have too much missing data

In [8]:
X_train.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1,inplace=True)
X_valid.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
#let us see the categorical columns
cat_col=[]
for col in X_train.columns:
    if(X_train[col].dtype==object):
        cat_col.append(col)
print(cat_col)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


let us also find the numeric coumns for future use

In [10]:
num_col=[]
num_col=list(X_train.drop(cat_col,axis=1).columns)
#we dont want taarget variable in feature sets

In [11]:
train[cat_col].nunique(axis=0,dropna=True)

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          8
Exterior1st      15
Exterior2nd      16
MasVnrType        4
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           6
HeatingQC         5
CentralAir        2
Electrical        5
KitchenQual       4
Functional        7
GarageType        6
GarageFinish      3
GarageQual        5
GarageCond        5
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64

# To encode the categorical data to numeric we will use Target encoder of category_encoders module 

Target encoder converts the categorical values into numerical values by calculating the probability of target variable for the repective categories

In [12]:
import category_encoders as ce
tar_encoder= ce.TargetEncoder(cols= cat_col)
tar_encoder.fit(X_train[cat_col],y_train)
X_train= X_train.join(tar_encoder.transform(X_train[cat_col]).add_suffix("_te"))
X_valid= X_valid.join(tar_encoder.transform(X_valid[cat_col]).add_suffix("_te"))

In [13]:
X_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,Electrical_te,KitchenQual_te,Functional_te,GarageType_te,GarageFinish_te,GarageQual_te,GarageCond_te,PavedDrive_te,SaleType_te,SaleCondition_te
930,931,20,RL,73.0,8925,Pave,IR1,HLS,AllPub,Inside,...,187320.733445,213391.308429,183618.068684,202501.549872,239303.034810,187768.181664,188346.234454,186926.069109,173346.078671,175368.825368
656,657,20,RL,72.0,10007,Pave,IR1,Lvl,AllPub,Inside,...,187320.733445,213391.308429,183618.068684,202501.549872,202789.618799,187768.181664,188346.234454,186926.069109,173346.078671,175368.825368
45,46,120,RL,61.0,7658,Pave,Reg,Lvl,AllPub,Inside,...,187320.733445,324619.989011,183618.068684,202501.549872,202789.618799,187768.181664,188346.234454,186926.069109,173346.078671,175368.825368
1348,1349,20,RL,NaN,16196,Pave,IR3,Low,AllPub,Inside,...,187320.733445,213391.308429,183618.068684,202501.549872,202789.618799,187768.181664,188346.234454,186926.069109,173346.078671,175368.825368
55,56,20,RL,100.0,10175,Pave,IR1,Lvl,AllPub,Inside,...,187320.733445,140142.552711,183618.068684,202501.549872,202789.618799,187768.181664,188346.234454,186926.069109,173346.078671,175368.825368


For an instance we will use simple imputer to fill missing values

In [14]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(strategy='median')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train.drop(cat_col,axis=1)))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid.drop(cat_col,axis=1)))
imputed_X_train.columns=X_train.drop(cat_col,axis=1).columns
imputed_X_valid.columns=X_valid.drop(cat_col,axis=1).columns

In [15]:
# let us create a xgbclassifier model to fit the data and validate it
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as mae
reg=XGBRegressor(max_depth= 6, learning_rate=0.05,n_estimators=5000, n_jobs=-1,early_stopping_rounds=40)
reg.fit(imputed_X_train,y_train)
pred=reg.predict(imputed_X_valid)
print(mae(pred,y_valid))

16711.697988013697


In [16]:
test.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

In [17]:
# let us see our accuracy on submission
# first we need to target encode and remove missing values from test data
X_test= test.join(tar_encoder.transform(test[cat_col]).add_suffix("_te")).drop(cat_col,axis=1)
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))
imputed_X_test.columns=X_test.columns
X_test['SalePrice']=reg.predict(imputed_X_test)

In [18]:
X_test[['Id','SalePrice']].to_csv("submission.csv",index=False)

we achived a score of 0.135 with model

let us use the other methods of feature engineering and see the performance of model

# we will use one hot encoding to change categorical data

In [19]:
X_train[cat_col].nunique()

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        7
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          8
Exterior1st      15
Exterior2nd      16
MasVnrType        4
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           6
HeatingQC         5
CentralAir        2
Electrical        5
KitchenQual       4
Functional        7
GarageType        6
GarageFinish      3
GarageQual        5
GarageCond        5
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64

In [20]:
test[cat_col].nunique()

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         1
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        5
BldgType          5
HouseStyle        7
RoofStyle         6
RoofMatl          4
Exterior1st      13
Exterior2nd      15
MasVnrType        4
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           4
HeatingQC         5
CentralAir        2
Electrical        4
KitchenQual       4
Functional        7
GarageType        6
GarageFinish      3
GarageQual        4
GarageCond        5
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64

we see that columns Neighbourhood, Exterior1st and 2nd have high no. of categories so we will not one hot encode these instead we will target encode them

In [21]:
OH_col=cat_col.copy()
OH_col.remove('Neighborhood')
OH_col.remove('Exterior1st')
OH_col.remove('Exterior2nd')
OH_col

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [22]:
imp_X_train=pd.DataFrame()
imp_X_valid=pd.DataFrame()

my_imputer = SimpleImputer(strategy='median')
imp_X_train[num_col]=pd.DataFrame(my_imputer.fit_transform(X_train[num_col]))
imp_X_valid[num_col]=pd.DataFrame(my_imputer.transform(X_valid[num_col]))

my_imputer = SimpleImputer(strategy='most_frequent')
imp_X_train[cat_col]=pd.DataFrame(my_imputer.fit_transform(X_train[cat_col]))
imp_X_valid[cat_col]=pd.DataFrame(my_imputer.transform(X_valid[cat_col]))


In [23]:
imp_X_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,931.0,20.0,73.0,8925.0,8.0,5.0,2007.0,2007.0,0.0,16.0,...,SBrkr,Gd,Typ,Attchd,Fin,TA,TA,Y,WD,Normal
1,657.0,20.0,72.0,10007.0,5.0,7.0,1959.0,2006.0,54.0,806.0,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,46.0,120.0,61.0,7658.0,9.0,5.0,2005.0,2005.0,412.0,456.0,...,SBrkr,Ex,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,1349.0,20.0,69.0,16196.0,7.0,5.0,1998.0,1998.0,0.0,1443.0,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
4,56.0,20.0,100.0,10175.0,6.0,5.0,1964.0,1964.0,272.0,490.0,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


In [24]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_train = pd.DataFrame(OH_encoder.fit_transform(imp_X_train[OH_col]))
OH_valid = pd.DataFrame(OH_encoder.transform(imp_X_valid[OH_col]))

# One-hot encoding removed index, we need to put it back
OH_train.index = X_train.index
OH_valid.index = X_valid.index
#we need to add the numeric columns
OH_train=OH_train.join(imp_X_train[num_col])
OH_valid=OH_valid.join(imp_X_valid[num_col])

#we need add the three categorical cols we left as target encoded
L=['Neighborhood','Exterior1st','Exterior2nd']
tar_encoder= ce.TargetEncoder(cols= L)
tar_encoder.fit(imp_X_train[L],y_train)
OH_train= OH_train.join(tar_encoder.transform(imp_X_train[L],y_train))
OH_valid= OH_valid.join(tar_encoder.transform(imp_X_valid[L]))

In [25]:
OH_train.shape

(1314, 217)

Model training and prediction

In [26]:
reg=XGBRegressor(max_depth= 30, learning_rate=0.05,n_estimators=5000, n_jobs=-1,early_stopping_rounds=40)
reg.fit(OH_train,y_train)
pred=reg.predict(OH_valid)
print(mae(pred,y_valid))

31958.027771832192


This didnt worked well we will try another approach and parameter tuning to get better results 